In [43]:
import pandas_ta as ta
import pandas as pd
import ccxt
import plotly as plt
import plotly.graph_objects as go
import vectorbt as vb
import numpy
import warnings

In [44]:
exchange=ccxt.binance({
    "api_key":"kVxc7iv6jJaTiXG4iPlVSOHvESaFs8mndYwW8vyf9ygBu7XaatkIVsbpBtOpUuRD",
    "secret":"ophoMQWGuGCbT8RYMm6R1SvsYyC1zlBHgxBBiVVjEvkbtSNHnuoRCOSjNj4urKO0",
    "enableRateLimit":True,
})
exchange.load_markets()

{'ETH/BTC': {'id': 'ETHBTC',
  'symbol': 'ETH/BTC',
  'base': 'ETH',
  'quote': 'BTC',
  'baseId': 'ETH',
  'quoteId': 'BTC',
  'active': True,
  'type': 'spot',
  'linear': None,
  'inverse': None,
  'spot': True,
  'swap': False,
  'future': False,
  'option': False,
  'margin': True,
  'contract': False,
  'contractSize': None,
  'expiry': None,
  'expiryDatetime': None,
  'optionType': None,
  'strike': None,
  'settle': None,
  'settleId': None,
  'precision': {'amount': 4, 'price': 6, 'base': 8, 'quote': 8},
  'limits': {'leverage': {'min': None, 'max': None},
   'amount': {'min': 0.0001, 'max': 100000.0},
   'price': {'min': 1e-06, 'max': 922327.0},
   'cost': {'min': 0.0001, 'max': None},
   'market': {'min': 0.0, 'max': 1846.44461243}},
  'info': {'symbol': 'ETHBTC',
   'status': 'TRADING',
   'baseAsset': 'ETH',
   'baseAssetPrecision': '8',
   'quoteAsset': 'BTC',
   'quotePrecision': '8',
   'quoteAssetPrecision': '8',
   'baseCommissionPrecision': '8',
   'quoteCommissionP

In [45]:
symbol="BTC/USDT"
timeframe="1d"

candles=exchange.fetch_ohlcv(symbol,timeframe=timeframe,limit=None)
df=pd.DataFrame(candles[:],columns=["timestamp","open","high","low","close","volume"])
df["dt"]=pd.to_datetime(df["timestamp"],unit="ms")

In [46]:
df

,timestamp,open,high,low,close,volume,dt
0,1608768000000,23232.39,23794.43,22703.42,23729.20,69013.834252,2020-12-24
1,1608854400000,23728.99,24789.86,23433.60,24712.47,79519.943569,2020-12-25
2,1608940800000,24712.47,26867.03,24500.00,26493.39,97806.513386,2020-12-26
3,1609027200000,26493.40,28422.00,25700.00,26281.66,148455.586214,2020-12-27
4,1609113600000,26281.54,27500.00,26101.00,27079.41,79721.742496,2020-12-28
...,...,...,...,...,...,...,...
495,1651536000000,38525.16,38651.51,37517.80,37728.95,40316.453580,2022-05-03
496,1651622400000,37728.95,40023.77,37670.00,39690.00,62574.617360,2022-05-04
497,1651708800000,39690.00,39845.51,35571.90,36552.97,88722.433550,2022-05-05
498,1651795200000,36552.97,36675.63,35258.00,36013.77,68437.801870,2022-05-06


In [47]:
#df=pd.read_csv("tutorial3.csv")

In [48]:
df['ema']=round(ta.ema(df['close'],length=200),2)

In [49]:
entries=df.apply(lambda x:(x['close']<x['ema']),axis=1)
exits=df.apply(lambda x:(x['close']>x['ema']),axis=1)
pf=vb.Portfolio.from_signals(df['close'],entries=entries,exits=exits,init_cash=10000,fees=0.001)

In [50]:
warnings.simplefilter(action='ignore',category=FutureWarning)
pf.stats()

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\120777\OneDrive\桌面\tutorial\venv3.8\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                             0.000000
End                             499.000000
Period                          500.000000
Start Value                   10000.000000
End Value                     11592.937058
Total Return [%]                 15.929371
Benchmark Return [%]             51.386225
Max Gross Exposure [%]          100.000000
Total Fees Paid                 178.328926
Max Drawdown [%]                 30.896670
Max Drawdown Duration           132.000000
Total Trades                      7.000000
Total Closed Trades               6.000000
Total Open Trades                 1.000000
Open Trade PnL                -2352.909334
Win Rate [%]                     83.333333
Best Trade [%]                   24.813295
Worst Trade [%]                  -7.832537
Avg Winning Trade [%]             8.984290
Avg Losing Trade [%]             -7.832537
Avg Winning Trade Duration       11.200000
Avg Losing Trade Duration        90.000000
Profit Factor                     4.333047
Expectancy 

In [53]:
fig=go.Figure(data=[go.Candlestick( x=df['dt'],
                                    open=df['open'],
                                    high=df['high'],
                                    low=df['low'],
                                    close=df['close'],
                                    name="k棒"),
                                    
                    go.Scatter     (x=df['dt'],
                                    y=df['ema'],
                                    line=dict(color='black',width=2),
                                    name="200 ema")
                    ]
)

fig.update_layout(  xaxis_rangeslider_visible=False,
                    title="BTC 1DAY with ema",)
fig.show()